In [49]:
#@title Invoke What-If Tool for test data and the trained model {display-mode: "form"}

num_datapoints = 2000  #@param {type: "number"}
tool_height_in_px = 1000  #@param {type: "number"}

from witwidget.notebook.visualization import WitConfigBuilder
from witwidget.notebook.visualization import WitWidget

In [50]:
import pandas as pd
import tensorflow as tf
import numpy as np

In [52]:
model_path = "../networks/Trained/hopeful/model_dir_647_0/"
my_estimator = tf.estimator.Estimator(model_fn=test_model_fn,
                                      model_dir=model_path,
                                     )

INFO:tensorflow:Using default config.


I0823 18:15:56.328639 140462434420544 estimator.py:1739] Using default config.


INFO:tensorflow:Using config: {'_model_dir': '../networks/Trained/hopeful/model_dir_647_0/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fbf0c8aacc0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0823 18:15:56.330549 140462434420544 estimator.py:201] Using config: {'_model_dir': '../networks/Trained/hopeful/model_dir_647_0/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fbf0c8aacc0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


W0823 18:15:56.332128 140462434420544 estimator.py:1924] Estimator's model_fn (<function test_model_fn at 0x7fbefaa29158>) includes params argument, but params are not passed to Estimator.


In [53]:
FEATURES_NAME = ['Guess 1', 'Guess 2', 'Guess 3', 'Guess 4', 'Guess 5',
          'Guess 6', 'Guess 7', 'Guess 8', 'Guess 9', 'Guess 10']

TARGET_NAME = 'PASSWORD'

features_and_labels = FEATURES_NAME + [TARGET_NAME]

In [54]:
# Optimize the df load
feature_type = {}
cuts_type = {}
feature_type[TARGET_NAME] = str
for feature in FEATURES_NAME:
    feature_type[feature] = str
    cuts_type[feature] = int
    
path_df_train = "../database/dbML/hopeful/JOINED_hopeful_2019-07-01_0.7_train.csv"
df_train = pd.read_csv(path_df_train, delimiter=',',encoding='utf-8', skip_blank_lines=True, dtype=feature_type)

In [55]:
# Creates a tf feature spec from the dataframe and columns specified.
def create_feature_spec(df, columns=None):
    feature_spec = {}
    if columns == None:
        columns = df.columns.values.tolist()
    for f in columns:
        if df[f].dtype is np.dtype(np.int64):
            feature_spec[f] = tf.FixedLenFeature(shape=(), dtype=tf.int64)
        elif df[f].dtype is np.dtype(np.float64):
            feature_spec[f] = tf.FixedLenFeature(shape=(), dtype=tf.float32)
        else:
            feature_spec[f] = tf.FixedLenFeature(shape=(), dtype=tf.string)
    return feature_spec

In [56]:
# Create a feature spec for the classifier
feature_spec = create_feature_spec(df_train, features_and_labels)

In [57]:
# Converts a dataframe into a list of tf.Example protos.
def df_to_examples(df, columns=None):
    examples = []
    if columns == None:
        columns = df.columns.values.tolist()
    for index, row in df.iterrows():
        example = tf.train.Example()
        for col in columns:
            if df[col].dtype is np.dtype(np.int64):
                example.features.feature[col].int64_list.value.append(int(row[col]))
            elif df[col].dtype is np.dtype(np.float64):
                example.features.feature[col].float_list.value.append(row[col])
            elif row[col] == row[col]:
                example.features.feature[col].bytes_list.value.append(row[col].encode('utf-8'))
        examples.append(example)
    return examples

In [58]:
num_datapoints = 2000  
tool_height_in_px = 1000  

test_examples = df_to_examples(df_train[0:num_datapoints])

In [59]:
# Setup the tool with the test examples and the trained classifier
config_builder = WitConfigBuilder(test_examples).set_estimator_and_feature_spec(classifier, feature_spec)

WitWidget(config_builder, height=tool_height_in_px)

WitWidget(config={'model_type': 'classification', 'label_vocab': [], 'are_sequence_examples': False, 'inferenc…